In [ ]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
url = os.getenv("MONGO_URI")
client = MongoClient(url)
db = client.get_database("kuwtk")

In [ ]:
c_wiki_episodes = db["wiki_episodes"]
c_imbd_episodes = db["imbd_episodes"]
c_imbd_episodes_fullcredits = db["imbd_episodes_fullcredits"]

In [ ]:
we = [d for d in c_wiki_episodes.find()]
ie = [d for d in c_imbd_episodes.find()]
ief = [d for d in c_imbd_episodes_fullcredits.find()]


In [ ]:
season = 0
for e in we:
    if e["episode"] == 1:
        season += 1
    e["season"] = season
    e["code"] = f"S{e['season']}E{e['episode']}"
we_dt = pd.DataFrame(we)
we_dt.drop(columns=["_id"], inplace=True)
we_dt

In [ ]:
for e in ie:
    e["code"] = f"S{e['season']}E{e['episode']}"
ie_dt = pd.DataFrame(ie)
ie_dt.drop(columns=["_id"], inplace=True)


In [ ]:
dt2 = ie_dt.set_index('code').join(we_dt.set_index('code'),lsuffix='_ie', rsuffix='_we')

dt2.dropna(subset=["episode_overall"],inplace=True)
dt2.sort_values('episode_overall')

In [ ]:
d4 = dt2.drop(columns=["watch_links","raw_air_date","air_date_ie","episode_we","season_we","title_we"])
d4.info()

In [ ]:
d4.rename(columns={
    "episode_ie":"episode",
    "season_ie":"season",
    "title_ie":"title",
    "air_date_we":"air_date"},inplace=True)
d5 = d4.astype({"episode_overall":int})
d5["description"] = d5["description"].apply(lambda x: x.strip())
d5["title"] = d5["title"].apply(lambda x: x.strip())

In [ ]:
docs = d5.to_json(orient="records")
parsed = json.loads(docs)

In [ ]:
coll_episodes = db["episodes"]
res = coll_episodes.insert_many(parsed)
res